# Visibility

## Table of Contents

1. [Time](#time)
   1. [First Cell](#first-cell)
   2. [Second Cell](#second-cell)

## Time

### First Cell

Import required packages 

In [1]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt


### Second Cell

Implementation of the function `JulianDate()`: it takes the date in format `[YY;MM;DD;hh;mm;ss]` and gives the same date in Julian days `[DD;hh;mm;ss]`

In [ ]:

##* 
def JulianDate(date):
	"""Function to convert solar date to Julian days
	
	:type date: list
	:param date: solar date

	:rtype: list
	:return: julian day
	"""
	juliandate = []
	return juliandate
##* 


In [ ]:

##*
def ConvAngle(angle):
	"""Convert angles from degrees to radiants and vice versa

	Parameters
	----------
	angle : list or float
		value of the angle to convert
		- degrees : the variable is a list [dd, mm, ss.ss]
		- radiant : the variable is a float 

	Returns
	-------
	cangle : float or list
		converted angle
		- radiant : if `angle` is in degrees. Variable is a float
		- degrees : if `angle` is in radiant. Variable is a list
	
	Examples
	--------
	>>> ConvAngle([90, 0., 0.])/pi
	0.5
	>>> ConvAngle(pi/2)
	[90, 0., 0.]	
	"""
	alpha = pi/180
	if(type(angle) == list):
		cangle = angle[0] + angle[1]/60 + angle[2]/3600
		cangle *= alpha
	else:
		angle /= alpha
		cangle = np.zeros(3)
		cangle[0] = int(angle)
		cangle[1] = int((angle - cangle[0])*60)
		cangle[2] = ((angle-cangle[0])*60 - cangle[1])*60
	return cangle
##*